# Plot x for fixed y & z

In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
TIME_LIM = 30  # seconds or time steps

In [2]:
def varyX(minX, maxX, y, z, timeLimit=30):
    xs = range(minX, maxX+1)
    probs = {}
    
    for x in xs:
        state = (x,y,z)
        # Set up signal handler for SIGALRM, saving previous value
        oldHandler = signal.signal(signal.SIGALRM, sigalarmHandler)
        # Start timer
        signal.alarm(TIME_LIM)
        try:
            probs[x] = LoserAnalysis(state).getHittingProb()
        except TimeoutException:
            print(f'Took too long to find the hitting prob for state {state}')
        except RecursionError:
            print(f'maximum recursion depth exceeded for state {state}')
        finally:
            # Turn off timer
            signal.alarm(0)
            # Restore handler to previous value
            signal.signal(signal.SIGALRM, oldHandler)
        
    # use numerical approximation for the unrecorded states
    missedStates = [(x,y,z) for x in xs if x not in probs]
    numericalProbs = {s[0]:LoserAnalysis(s).getHittingProbNumerical() for s in missedStates}
    
    return probs, numericalProbs

In [3]:
# Automate the process
def plotXforFixedYZ(minX, maxX, y, z, probFunction=varyX, t=30):
    """
    Plots x values within the range [`minX`, `maxX`] (inclusive) for fixed y and z.
    The `timeLimit` (in seconds) casts a limit on the computation time of the hitting prob for each state.
    Saves the plot to a file provided by the string `figName`.
    """
    exactProbs, numericalProbs = probFunction(minX, maxX, y, z, t)
    
    # plot
    plt.scatter([x for x in exactProbs.keys()], exactProbs.values(), 
            color='b', label='exact probs', s=2)
    plt.scatter([x for x in numericalProbs.keys()], numericalProbs.values(), 
                color='r', label='numerical probs', s=2)

    # add graph features
    plt.xlabel('x')
    plt.ylabel('P(loser = Player 1)')
    plt.title(f"Hitting probabilities for different x values (y = {y}, z = {z})")
    plt.legend()

    plt.savefig(f'Hitting probs for different x (y = {y}, z = {z}) - {str(probFunction)}, game1.png')
    plt.show()

In [4]:
class TimeoutException(Exception):
    """
    Custom exception for timeout.
    """
    pass

def sigalarmHandler(signum, frame):
    """
    Handler function to be called when SIGALRM is received.
    """
    # We get signal!
    raise TimeoutException()